In [1]:
import pandas as pd
import numpy as np
import seaborn as sn
from suntime import Sun
import pytz
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score
from xgboost import XGBRegressor
import optuna
pd.set_option('display.float_format', lambda x: '%.8f' % x)

# Helper Functions

In [2]:
def datetime_index(dataframe):
    dataframe.columns = dataframe.columns.str.replace(' ', '')
    dataframe['Datetime'] = pd.to_datetime(dataframe['DateTime'])
    dataframe = dataframe.set_index(['Datetime'])
    del dataframe['DateTime']
    return dataframe

def fill_na(dataframe):
    dataframe['WWCode'] = dataframe['WWCode'].fillna(0).astype(int)
    return dataframe

def day_night(dataframe):

    coordinates = [40.239, 33.029] # Ankara
    dataframe['datetime_col'] = dataframe.index
    
    dataframe["Date"] = dataframe['datetime_col'].apply(pd.to_datetime).dt.date.apply(str)
    dataframe["Hour"] = dataframe['datetime_col'].apply(pd.to_datetime).dt.hour
    dataframe["Day_of_Week"] = dataframe['datetime_col'].apply(pd.to_datetime).dt.dayofweek
    dataframe["Quarter"] = dataframe['datetime_col'].apply(pd.to_datetime).dt.quarter
    dataframe["Month"] = dataframe['datetime_col'].apply(pd.to_datetime).dt.month
    dataframe["Year"] = dataframe['datetime_col'].apply(pd.to_datetime).dt.year
    dataframe["Day_of_Year"] = dataframe['datetime_col'].apply(pd.to_datetime).dt.dayofyear
    dataframe["Week"] = dataframe['datetime_col'].apply(pd.to_datetime).dt.week
    dataframe["Week_of_Year"] = dataframe['datetime_col'].apply(pd.to_datetime).dt.weekofyear
    
    sun = Sun(coordinates[0], coordinates[1])
    tz =pytz.timezone('Europe/Istanbul')
    dataframe["Day"]= dataframe[["Date","Hour"]].apply(lambda x : sun.get_local_sunrise_time(pd.to_datetime(x["Date"]).
                                                                date(),local_time_zone=tz).
                                         hour <= x["Hour"] <= sun.get_local_sunset_time(pd.to_datetime(x["Date"]).
                                                                date(),local_time_zone=tz).hour ,axis=1)
    dataframe = dataframe.drop(['Date', 'datetime_col'], axis = 1)
    return dataframe

def wwcode_encoder(dataframe):
    for index, row in dataframe.iterrows():
        if row['WWCode'] < 10:
            dataframe.loc[index, 'WWCode'] = 1
        elif row['WWCode'] < 20:
            dataframe.loc[index, 'WWCode'] = 2
        elif row['WWCode'] < 30:
            dataframe.loc[index, 'WWCode'] = 3
        elif row['WWCode'] < 40:
            dataframe.loc[index, 'WWCode'] = 4
        elif row['WWCode'] < 50:
            dataframe.loc[index, 'WWCode'] = 5
        elif row['WWCode'] < 60:
            dataframe.loc[index, 'WWCode'] = 6
        elif row['WWCode'] < 70:
            dataframe.loc[index, 'WWCode'] = 7
        elif row['WWCode'] < 80:
            dataframe.loc[index, 'WWCode'] = 8
        elif row['WWCode'] < 90:
            dataframe.loc[index, 'WWCode'] = 9
        elif row['WWCode'] < 100:
            dataframe.loc[index, 'WWCode'] = 10
    return dataframe
    
    
def season_encoder(dataframe):
    dataframe['Season'] = 0
    for index, row in dataframe.iterrows():
        if row['Month'] < 3 or row['Month'] == 12:
            dataframe.loc[index, 'Season'] = 1
        elif row['Month'] < 6:
            dataframe.loc[index, 'Season'] = 2
        elif row['Month'] < 9:
            dataframe.loc[index, 'Season'] = 3
        elif row['Month'] < 12:
            dataframe.loc[index, 'Season'] = 4
    return dataframe

def fig_plot():
    plt.figure(figsize = (10,15))
    plt.scatter(Y_test, pred_val)
    plt.xlabel('Actual')
    plt.ylabel('Predicted')
    
    
def objective(trial):
    learning_rate = trial.suggest_float("learning_rate", 1e-2, 0.25, log=True)
    reg_lambda = trial.suggest_loguniform("reg_lambda", 1e-8, 100.0)
    reg_alpha = trial.suggest_loguniform("reg_alpha", 1e-8, 100.0)
    subsample = trial.suggest_float("subsample", 0.1, 1.0)
    colsample_bytree = trial.suggest_float("colsample_bytree", 0.1, 1.0)
    max_depth = trial.suggest_int("max_depth", 1, 7)
    early_stopping_rounds = trial.suggest_int("early_stopping_rounds", 100, 500)
    n_estimators = trial.suggest_int("n_estimators", 0, 10000)
    
    model = XGBRegressor(
        random_state=50,
        tree_method="gpu_hist",
        gpu_id=0,
        predictor="gpu_predictor",
        n_estimators=n_estimators,
        learning_rate=learning_rate,
        reg_lambda=reg_lambda,
        reg_alpha=reg_alpha,
        subsample=subsample,
        colsample_bytree=colsample_bytree,
        max_depth=max_depth,
    )
    model.fit(
        X_train,
        Y_train,
        early_stopping_rounds=early_stopping_rounds,
        eval_set=[(X_test, Y_test)],
        verbose=1200,
    )
    preds_valid = model.predict(X_test)
    rmse = mean_squared_error(Y_test, preds_valid, squared=False)

    return rmse

# Data Extraction

In [3]:
submission = pd.read_csv('sample_submission.csv')
label = datetime_index(pd.read_csv('generation.csv', delimiter=';', decimal=','))
features = datetime_index(pd.read_csv('temperature.csv', delimiter=';', decimal=','))

In [4]:
x_train  = fill_na(features.loc['2019-01-01':'2021-11-30'])
x_test = day_night(fill_na(features.loc['2021-12-01':'2021-12-31']))
y_train  = label.loc['2019-01-01':'2021-11-30']

C:\Users\asus\AppData\Local\Temp/ipykernel_1880/2852319294.py:24: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  dataframe["Week"] = dataframe['datetime_col'].apply(pd.to_datetime).dt.week
C:\Users\asus\AppData\Local\Temp/ipykernel_1880/2852319294.py:25: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  dataframe["Week_of_Year"] = dataframe['datetime_col'].apply(pd.to_datetime).dt.weekofyear


In [5]:
df_train = day_night(pd.concat([ x_train, y_train], axis=1))

C:\Users\asus\AppData\Local\Temp/ipykernel_1880/2852319294.py:24: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  dataframe["Week"] = dataframe['datetime_col'].apply(pd.to_datetime).dt.week
C:\Users\asus\AppData\Local\Temp/ipykernel_1880/2852319294.py:25: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  dataframe["Week_of_Year"] = dataframe['datetime_col'].apply(pd.to_datetime).dt.weekofyear


### Preprocessing

In [6]:
df_train = wwcode_encoder(df_train)
df_train = season_encoder(df_train)

x_test = wwcode_encoder(x_test)
x_test = season_encoder(x_test)

In [7]:
# df_train.drop(['ComfortTemperature'], axis=1, inplace=True)
# x_test.drop(['ComfortTemperature'], axis=1, inplace=True)

In [8]:
# df_train

In [9]:
# df_train = pd.get_dummies(df_train, columns = ['WWCode'])
# x_test = pd.get_dummies(x_test, columns = ['WWCode'])

##### Dataset no:1

In [10]:
# X_train = df_train.loc['2019-01-01':'2021-10-31'].drop(['Generation'], axis=1)
# X_test = df_train.loc['2021-10-01':'2021-11-30'].drop(['Generation'], axis=1)
# Y_train = df_train.Generation.loc['2019-01-01':'2021-10-31']
# Y_test = df_train.Generation.loc['2021-10-01':'2021-11-30']

##### Dataset no:2

In [11]:
X_train = df_train.loc['2019-01-01':'2021-04-30'].drop(['Generation'], axis=1)
X_test = df_train.loc['2021-05-01':'2021-11-30'].drop(['Generation'], axis=1)
Y_train = df_train.Generation.loc['2019-01-01':'2021-04-30']
Y_test = df_train.Generation.loc['2021-05-01':'2021-11-30']

In [12]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=20)

[I 2022-02-28 17:26:00,174] A new study created in memory with name: no-name-37eb73e7-b271-4fa5-9d88-b91073990186


[0]	validation_0-rmse:180.58514
[1200]	validation_0-rmse:58.61502
[1941]	validation_0-rmse:53.91369


[I 2022-02-28 17:26:06,721] Trial 0 finished with value: 53.907205438094095 and parameters: {'learning_rate': 0.049372886961214835, 'reg_lambda': 1.0593097219317527, 'reg_alpha': 3.91489200049281e-08, 'subsample': 0.5183147227664918, 'colsample_bytree': 0.31454283056490673, 'max_depth': 1, 'early_stopping_rounds': 177, 'n_estimators': 1942}. Best is trial 0 with value: 53.907205438094095.


[0]	validation_0-rmse:180.25249
[1200]	validation_0-rmse:25.24310
[2400]	validation_0-rmse:23.31722
[2648]	validation_0-rmse:23.27770


[I 2022-02-28 17:26:19,383] Trial 1 finished with value: 23.24952284496195 and parameters: {'learning_rate': 0.0157765014356488, 'reg_lambda': 0.28872484271649423, 'reg_alpha': 0.0660100873373337, 'subsample': 0.27933118942292673, 'colsample_bytree': 0.6278031558590549, 'max_depth': 3, 'early_stopping_rounds': 101, 'n_estimators': 6307}. Best is trial 1 with value: 23.24952284496195.


...

[I 2022-02-28 17:36:37,509] Trial 17 finished with value: 21.354863840172335 and parameters: {'learning_rate': 0.026905165436331185, 'reg_lambda': 0.0062853150456468184, 'reg_alpha': 1.8781601553259078e-07, 'subsample': 0.6024597157802487, 'colsample_bytree': 0.9996798859599085, 'max_depth': 5, 'early_stopping_rounds': 429, 'n_estimators': 1424}. Best is trial 12 with value: 21.11394799195727.


[0]	validation_0-rmse:169.71541
[449]	validation_0-rmse:23.93617


[I 2022-02-28 17:37:04,408] Trial 18 finished with value: 22.64807098589384 and parameters: {'learning_rate': 0.08011999959722806, 'reg_lambda': 1.643446702057063e-07, 'reg_alpha': 2.2216927875763327e-05, 'subsample': 0.3837607757292846, 'colsample_bytree': 0.8396091964504382, 'max_depth': 7, 'early_stopping_rounds': 335, 'n_estimators': 6155}. Best is trial 12 with value: 21.11394799195727.


[0]	validation_0-rmse:176.56795
[1039]	validation_0-rmse:22.38165


[I 2022-02-28 17:37:44,791] Trial 19 finished with value: 22.323501577640325 and parameters: {'learning_rate': 0.06269806315706278, 'reg_lambda': 0.00013248512281752132, 'reg_alpha': 1.4235963899729177e-07, 'subsample': 0.8086567935450937, 'colsample_bytree': 0.5341624036609057, 'max_depth': 6, 'early_stopping_rounds': 485, 'n_estimators': 3856}. Best is trial 12 with value: 21.11394799195727.


In [13]:
optuna_params = study.best_params
optuna_params

{'learning_rate': 0.03547901178897394,
 'reg_lambda': 2.5347525209294996e-08,
 'reg_alpha': 1.1095508085972166e-08,
 'subsample': 0.6591068850848868,
 'colsample_bytree': 0.9738853786020008,
 'max_depth': 7,
 'early_stopping_rounds': 365,
 'n_estimators': 3323}

In [14]:
model = XGBRegressor(
    random_state=50,
    tree_method="gpu_hist",
    gpu_id=0,
    predictor="gpu_predictor",
    **optuna_params
)
model.fit(
    X_train,
    Y_train,
    early_stopping_rounds=optuna_params['early_stopping_rounds'],
    eval_set=[(X_test, Y_test)],
    verbose=1000,
)

[17:37:44] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:576: 
Parameters: { "early_stopping_rounds" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	validation_0-rmse:176.99229
[560]	validation_0-rmse:21.94708


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.9738853786020008,
             early_stopping_rounds=365, enable_categorical=False, gamma=0,
             gpu_id=0, importance_type=None, interaction_constraints='',
             learning_rate=0.03547901178897394, max_delta_step=0, max_depth=7,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=3323, n_jobs=8, num_parallel_tree=1,
             predictor='gpu_predictor', random_state=50,
             reg_alpha=1.1095508085972166e-08,
             reg_lambda=2.5347525209294996e-08, scale_pos_weight=1,
             subsample=0.6591068850848868, tree_method='gpu_hist',
             validate_parameters=1, verbosity=None)

In [15]:
pred_val_optuna = model.predict(X_test)
pred_val_optuna = pd.DataFrame(pred_val_optuna, index=X_test.index, columns=['pred'])

In [16]:
pred_val_optuna = day_night(pred_val_optuna)
pred_val_optuna.pred[pred_val_optuna.Day == False] = 0
pred_val_optuna = pred_val_optuna.pred
pred_val_optuna

C:\Users\asus\AppData\Local\Temp/ipykernel_1880/2852319294.py:24: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  dataframe["Week"] = dataframe['datetime_col'].apply(pd.to_datetime).dt.week
C:\Users\asus\AppData\Local\Temp/ipykernel_1880/2852319294.py:25: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  dataframe["Week_of_Year"] = dataframe['datetime_col'].apply(pd.to_datetime).dt.weekofyear
C:\Users\asus\AppData\Local\Temp/ipykernel_1880/1247003563.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pred_val_optuna.pred[pred_val_optuna.Day == False] = 0


Datetime
2021-05-01 00:00:00   0.00000000
2021-05-01 01:00:00   0.00000000
2021-05-01 02:00:00   0.00000000
2021-05-01 03:00:00   0.00000000
2021-05-01 04:00:00   0.00000000
                         ...    
2021-11-30 19:00:00   0.00000000
2021-11-30 20:00:00   0.00000000
2021-11-30 21:00:00   0.00000000
2021-11-30 22:00:00   0.00000000
2021-11-30 23:00:00   0.00000000
Name: pred, Length: 5136, dtype: float32

In [17]:
rmse_optuna = mean_squared_error(Y_test, pred_val_optuna, squared=False)
rmse_optuna

21.100847775631586